In [1]:
from pathlib import Path
import colorcet as cc
from osgeo import gdal
import numpy as np
import re
import glob
import copy
import numpy as np
import os, zipfile
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

from tkinter import *
from tkinter import ttk
import tkinter.font as font
import tkinter as tk
from tkinter import filedialog, messagebox
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from matplotlib import pyplot

from PIL import Image

In [5]:
raster_path = r"C:\Users\22566465\Desktop\TESTING_GEO_GRAPH\FortescueStitch_RTP.ers"
raster = gdal.Open(str(raster_path))

In [6]:
band = raster.GetRasterBand(1)
if band.GetMinimum() is None or band.GetMaximum() is None:
    band.ComputeStatistics(0)

In [ ]:
def array_transform(band):
    _array =  band.ReadAsArray().flatten()
    filt = band.ReadAsArray().flatten() != band.GetNoDataValue()
    
    # calc a stretch
    no_nan = band.ReadAsArray().flatten()[filt]

    quantile = QuantileTransformer(output_distribution='normal')
    data_trans = quantile.fit(no_nan.reshape(-1, 1))

    modified = np.array([data_trans.transform(x.reshape(-1, 1)) if x!= band.GetNoDataValue() else np.array((255,255,255),dtype='uint8') for x in _array])
    return(modified)


In [13]:

_array =  band.ReadAsArray()
ashape = _array.shape
flat_a = _array.flatten()

filt = band.ReadAsArray().flatten() != band.GetNoDataValue()
no_nan = band.ReadAsArray().flatten()[filt]
quantile = QuantileTransformer(output_distribution='normal')
data_trans = quantile.fit(no_nan.reshape(-1, 1))
trans = data_trans.transform(no_nan.reshape(-1, 1))

sel_map = plt.get_cmap('cet_CET_C2s')
colors = np.uint8(sel_map(trans, bytes=True))[:, :, :3]

output = []
counter = 0
for i,val in enumerate(filt):
    if val:
        output.append(colors[counter])
        counter += 1
    else:
        output.append(np.array([[255,255,255]],dtype='uint8'))
        
im_array = np.array(output).reshape(np.append(np.array(ashape),(3)))

image = Image.fromarray(im_array, mode="RGB")

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [ ]:
##-------------------------------------------------------GUI-FUNC----------------------------------------------------------##

class MainApplication(tk.Frame):
    def __init__(self, parent, v_numb, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent
        #variables
        self.selected_file=None
        self.saving_directory=None
        self.hill_shade = None
        self.color_map  = None
        
        self.raster     = None
        self.band       = None

        self.figure_1 = plt.figure(figsize=(10,6))
        #self.figure_2 = plt.figure(figsize=(10,6))
        self.v_numb = v_numb

        ###building ui
        ## setup
        self.plot_super     =   ttk.Labelframe(self.parent,text='Visuals',              width=1000, height=400,borderwidth= 3,relief="ridge")
        self.cont_frame     =   ttk.Labelframe(self.plot_super,text='Controls',             width=1000, height=100,borderwidth= 3,relief="ridge")
        self.plot_spati     =   ttk.Labelframe(self.plot_super,text='Spatial',width=400, height=400,borderwidth= 3,relief="ridge")
        self.plot_stats     =   ttk.Labelframe(self.plot_super,text='Histogram',width=400, height=400,borderwidth= 3,relief="ridge")
        
        self.canvas_1         =   FigureCanvasTkAgg(self.figure_1,master=self.plot_spati)
        #self.canvas_2         =   FigureCanvasTkAgg(self.figure_2,master=self.plot_stats)

        # packing
        self.plot_super.pack(side="left",fill="both",expand=True)
        self.cont_frame.pack(fill="both",side="bottom",expand=True)
        self.plot_spati.pack(fill="both",side="right",expand=True)
        self.plot_stats.pack(fill="both",side="right",expand=True)
        self.canvas_1.get_tk_widget().pack(fill="both",expand=True)
        #self.canvas_2.get_tk_widget().pack(fill="both",expand=True)

        ## import
        self.init_frame     =   ttk.Labelframe(self.cont_frame,text='Import Data',      width=300, height=100,borderwidth= 3,relief="ridge")
        self.load_selection =   tk.Button(self.init_frame, text = 'Select File',command=self.select_dir)
        self.confirm        =   tk.Button(self.init_frame, text = 'Load Selected File',command=self.import_dir)
        self.seldir_label   =   tk.Label(self.init_frame,fg="black", bg="white",text='No File Selected')
        # packing
        self.init_frame.pack(fill="x",side="left",expand=True)     
        self.load_selection.pack(fill="x",side="top",expand=True)   
        self.seldir_label.pack(fill="x",side="top",expand=True)        
        self.confirm.pack(fill="x",side="top",expand=True)            

        ## adjustment
        self.ajst_frame     =   ttk.Labelframe(self.cont_frame,text='Adjust Histogram',      width=400, height=100,borderwidth= 3,relief="ridge")
        self.cmap_frame     =   ttk.Labelframe(self.ajst_frame,text='Select Colormap',width=400, height=400,borderwidth= 3,relief="ridge")
        # packing
        self.ajst_frame.pack(fill="x",side="left",expand=True)     
        self.cmap_frame.pack(fill="x",side="top",expand=True)

        ## lets set some adjustments
        # our histogram lims dummy TODO do some nice calculation
        self.upper_init     = DoubleVar()
        self.lower_init     = DoubleVar()
        self.cmap_sel       = StringVar()

        color_map = [x for x in dir(cc.cm) if 'CET' in x]   #TODO generate list of options here

        # setting out default variables
        self.upper_init.set(0)
        self.lower_init.set(0)
        self.cmap_sel.set(color_map[0]) # default value

        # sliders
        self.upper_slider   =   tk.Scale(
            self.ajst_frame, from_=0, to=100,orient='horizontal',command=self.vis_process,label='Upper Bound', variable=self.upper_init
            )       

        self.lower_slider   =   tk.Scale(
            self.ajst_frame, from_=-100, to=0,orient='horizontal', command=self.vis_process,label='Lower Bound',variable=self.lower_init
            )

        # our option menus
        self.cmap_menu = tk.OptionMenu(self.cmap_frame, self.cmap_sel, *color_map,command=self.vis_process)

        # pack all these bad boys together
        self.upper_slider.pack(fill="x",expand=True)   
        self.lower_slider.pack(fill="x",expand=True)   
        self.cmap_menu.pack(fill="x",expand=True)

        ## save changes
        self.save_frame     =   ttk.Labelframe(self.cont_frame,text='Saving Outputs',   width=300, height=100,borderwidth= 3,relief="ridge")
        self.save_selection =   tk.Button(self.save_frame , text = 'Select Save directory',command=self.save_dir)
        self.savedir_label  =   tk.Label(self.save_frame,fg="black", bg="white",text='No Directory Selected')
        self.save           =   tk.Button(self.save_frame , text = 'Save Processed Data', command=self.save_processed)
        # packing
        self.save_frame.pack(fill="x",side="right",expand=True)     
        self.save_selection.pack(fill="x",side="bottom",expand=True) 
        self.savedir_label.pack(fill="x",side="bottom",expand=True)  
        self.save.pack(fill="x",side="bottom",expand=True)           

    def select_dir(self):
        f = filedialog.askopenfilename(filetypes=[("Rasters", ".ers .ERS")])

        if len(f) > 0:
            self.selected_file = f
            working_directory_split=self.selected_file.split('/')
            workdir_short='...  '+working_directory_split[-2]+'/'+working_directory_split[-1]
            self.seldir_label.configure(text=workdir_short)

        if len(f)==0:  # ask directory doens't return none, returns empty string ""
            self.selected_file = None
            self.raise_warining("No directory selected")

    def import_dir(self):

        if self.selected_file == None:
            self.raise_warining("No raster selected")

        if self.selected_file != None:
            self.raster = gdal.Open(str(self.selected_file))
            self.hill_shade = os.path.basename(self.selected_file)[:-4] + "_hillshade.tif"
            self.color_map  = os.path.basename(self.selected_file)[:-4] + "_colourmap.tif"
            band = self.raster.GetRasterBand(1)
            if band.GetMinimum() is None or band.GetMaximum() is None:
                band.ComputeStatistics(0)

            self.band = band.ReadAsArray()
            # set some reasonable values for the raster
            self.raise_warining("File Successfully Loaded")
            
    
    def vis_process(self,state):
        # TODO gotta put some plot stuff here

        ## Do some calulcations on the ERS

        
        # set a colormap variable 
        plt.get_cmap()

        #z_values = np.array()# for histogram limited by the min max or maybe just lines on hte hist etc

        # Plot it

        # stats
        figure_1 = self.figure_1
        plt.clf()
        plt.title('Distribution of Values')
        plt.hist(self.band,bins=50)
        self.canvas_1.draw()


        # fig=self.figure
        # plt.clf()
        # plt.subplot(2,1,1)
        # plt.title(name)
        # plt.plot(inrod_Do,inrod_Ga,label='In Rod',color='dimgrey')
        # plt.plot(ohole_Do,ohole_Ga,label='Open Hole',color='firebrick')
        # plt.xlabel('Depth - (m)')
        # plt.grid(True,which='major',axis='x')
        # plt.legend()
        # plt.subplot(2,1,2)
        # plt.plot(inrod_Dn,inrod_Ga,label='In Rod',color='dimgrey')
        # plt.plot(ohole_Do,ohole_Ga,label='Open Hole',color='firebrick')
        # plt.xlabel('Depth - (m)')
        # plt.grid(True,which='major',axis='x')
        # plt.legend()
        # plt.tight_layout()   
        # self.canvas.draw()

    def save_dir(self):
        f = filedialog.askdirectory()

        if len(f) > 0:
            self.saving_directory = f
            working_directory_split=self.saving_directory.split('/')
            workdir_short='...  '+working_directory_split[-2]+'/'+working_directory_split[-1]
            self.savedir_label.configure(text=workdir_short)

        if len(f)==0:  # ask directory doens't return none, returns empty string ""
            self.working_directory = None
            self.raise_warining("No directory selected")



    def save_processed(self):
        """
        using the data held in the program produces a set of las files and a report
        """
        # TODO gott a input plots etc
        
        ## Create hillshade file
        translate_options   = gdal.TranslateOptions(
            outputType      = gdal.gdalconst.GDT_UInt16,
            scaleParams     = [[band.GetMinimum(), band.GetMaximum(), 1, 65535]],
        )  # We leave 0 out so it should be used as a NaN indicator. ^u-int max min vals
        a = gdal.Translate(str(hillshade_file_path), raster, options=translate_options)


    def raise_warining(self,message):
        messagebox.showwarning("Warning",message)

    def raise_message(self,message):
        messagebox.Message(message)
    
if __name__ == "__main__":
    root = tk.Tk()
    v_numb = "0.1"
    gui = MainApplication(root,v_numb).pack(side="top", fill="both", expand=True)
    root.title('Earth Blender Data Prep - '+v_numb)
    root.resizable(True, True)
    root.mainloop()